In [ ]:
from io import BytesIO
import json
import time
from copy import deepcopy

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    ImageMessage,
    FlexMessage,
    FlexContainer,
    QuickReply,
    QuickReplyItem,
    MessageAction
)
from linebot.v3.webhooks import (
    MessageEvent,
    PostbackEvent,
    FollowEvent,
    UnfollowEvent,
    TextMessageContent,
    ImageMessageContent,
    LocationMessageContent
)
import cwa
from PIL import Image
from google.cloud import storage, firestore


with open('env.json', encoding='utf-8') as f:
    env = json.load(f)


storage_client = storage.Client.from_service_account_json('service.json')
bucket = storage_client.bucket('myfirstprojectnumberonebucket')

firestore_client = firestore.Client.from_service_account_json('service.json')
collection = firestore_client.collection('enos1')

app = Flask(__name__)

configuration = Configuration(access_token=env.get('CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(env.get('CHANNEL_SECRET'))

rich1 = 'richmenu-113ebbf2fe7b01ad064b393d6b8791fb'
rich2 = 'richmenu-15981903ae8a6ccc590b8b07e7881293'
navall = 'richmenu-5e5fd3ecae4f0ee760b4f885e065bd3d'  # navall.json
fans = 'richmenu-663c6b997ba713a8c2c33d7c0f672fd4'  # fans.json

users = []
pos = 0

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    print(event.message.text)  #
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    messages = []
    
    ask = event.message.text
    ask_map = {'hello': '我很好', 'hi': '您哪位'}
    ans = ask_map.get(ask)
    if not ans:
        if ask == 'me':  # me handling
            with open('carousel.json', encoding='utf-8') as f:
                flex_dict = json.load(f)
            flex_message = FlexMessage(altText='me', contents=FlexContainer.from_dict(flex_dict))
            messages.append(flex_message)
        elif ask == 'action':  # action handling
            with open('action.json', encoding='utf-8') as f:
                flex_dict = json.load(f)
            flex_message = FlexMessage(altText='action', contents=FlexContainer.from_dict(flex_dict))
            messages.append(flex_message)
        elif ask in '12345':
            n = int(ask)
            with open('me.json', encoding='utf-8') as f:
                me_dict = json.load(f)

            new_dicts = []
            count = 0
            for u in collection.stream():
                new_dict = deepcopy(me_dict)
                u = u.to_dict()
                new_dict['hero']['url'] = u['picture_url']
                new_dict['body']['contents'][0]['contents'][0]['contents'][1]['text'] = u['user_id']
                new_dict['body']['contents'][0]['contents'][1]['contents'][1]['text'] = u['display_name']
                new_dict['body']['contents'][0]['contents'][2]['contents'][1]['text'] = u['status_message'] or 'N/A'
                new_dicts.append(new_dict)
                count += 1
                if count >= n:
                    break

            car = {"type": "carousel", "contents": new_dicts}
            flex_message = FlexMessage(altText=f'{n} me', contents=FlexContainer.from_dict(car))
            messages.append(flex_message)
        else:  # weather
            ans = cwa.cwa2(ask, env.get('CWA_KEY'))
            ans = cwa.tostr(ans, '\n') or '無此站'
            
            if ans != '無此站':
                collection.document(event.source.user_id).update({'weather_count': firestore.Increment(1)})

            item1 = QuickReplyItem(action=MessageAction(label='文化大學', text='文化大學'))
            item2 = QuickReplyItem(action=MessageAction(label='陽明山', text='陽明山'))
            items = [item1, item2]
            quick_reply = QuickReply(items=items)

            messages.append(TextMessage(text=ans, quick_reply=quick_reply))
    else:  # hello
        messages.append(TextMessage(text=ans))
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=messages
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_message(event):
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    print(event.message.latitude)
    print(event.message.longitude)
    site = (event.message.latitude, event.message.longitude)
    ans = cwa.cwa2(site, env.get('CWA_KEY'))
    ans = cwa.tostr(ans, '\n') or '無此站'
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_content_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
        image = Image.open(BytesIO(message_content))
        text = f'{image.height} X {image.width}'

        # upload image
        blob_name = f'{event.source.user_id}_{event.message.id}'
        blob = bucket.blob(blob_name)
        blob.upload_from_string(message_content, content_type='image/jpeg')
        url = blob.public_url
        url = blob.generate_signed_url(int(time.time())+60)
        image_message = ImageMessage(original_content_url=url,
                                     preview_image_url=url)
        
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    TextMessage(text=text),
                    image_message
                ]
            )
        )


@handler.add(PostbackEvent)
def handle_message(event):
    global users, pos
    print(event.source.user_id)
    print(event.postback.data)
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if event.postback.data == 'nav':
            line_bot_api.link_rich_menu_id_to_user(event.source.user_id, navall)
            users = [u.to_dict() for u in collection.stream()]
            u = users[0]
            pos = 0
            # find first
        elif event.postback.data == 'left':
            pos = pos - 1
            if pos < 0:
                pos = 0
            u = users[pos]
        elif event.postback.data == 'right':
            pos = pos + 1
            if pos >= len(users):
                pos = len(users) - 1
            u = users[pos]
        elif event.postback.data == 'hello':
            line_bot_api.link_rich_menu_id_to_user(event.source.user_id, rich2)
        elif event.postback.data == 'return':
            #line_bot_api.link_rich_menu_id_to_user(event.source.user_id, rich1)
            line_bot_api.unlink_rich_menu_id_from_user(event.source.user_id)
            users = []
            
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=f'i see {event.postback.data}')]
            )
        )

@handler.add(FollowEvent)
def handle_message(event):
    print(event.source.user_id)
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        profile = dict(line_bot_api.get_profile(event.source.user_id))
        display = profile.get('display_name') or 'NA'
        welcome = f'Welcome {display}'

        collection.document(event.source.user_id).set(profile | {'follow': time.strftime('%Y/%m/%d-%H:%M:%S')}, merge=True)
        
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=welcome)]
            )
        )

@handler.add(UnfollowEvent)
def handle_message(event):
    print(event.source.user_id)
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        collection.document(event.source.user_id).update({'unfollow': time.strftime('%Y/%m/%d-%H:%M:%S')})


if __name__ == "__main__":
    app.run(port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


U091e03d020e37cce02ba9fd2f03377ec
nav


127.0.0.1 - - [22/Nov/2025 16:52:39] "POST /callback HTTP/1.1" 200 -


U091e03d020e37cce02ba9fd2f03377ec
return


...

127.0.0.1 - - [22/Nov/2025 16:52:48] "POST /callback HTTP/1.1" 200 -


U091e03d020e37cce02ba9fd2f03377ec
left


127.0.0.1 - - [22/Nov/2025 16:52:51] "POST /callback HTTP/1.1" 200 -


U091e03d020e37cce02ba9fd2f03377ec
right


127.0.0.1 - - [22/Nov/2025 16:52:52] "POST /callback HTTP/1.1" 200 -


In [5]:
# force unlink personal rich menu
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    line_bot_api.unlink_rich_menu_id_from_user('U091e03d020e37cce02ba9fd2f03377ec')

In [7]:
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)
    profile = line_bot_api.get_profile('U091e03d020e37cce02ba9fd2f03377ec')

In [12]:
dict(profile)

{'display_name': 'Enos',
 'user_id': 'U091e03d020e37cce02ba9fd2f03377ec',
 'picture_url': 'https://sprofile.line-scdn.net/0hRChPEC5sDWxFKSb35qhzEzV5DgZmWFR-bxxGCScvVg9xGEs5O0ZHWiQgUlgtGk07Pk4WCiN8Bl1JOnoKW3_xWEIZU1t_GUo8YUlLiw',
 'status_message': None,
 'language': 'zh-TW'}

In [ ]:
!pip install google-cloud-firestore

In [16]:
from google.cloud import firestore

firestore_client = firestore.Client.from_service_account_json('service.json')
collection = firestore_client.collection('enos1')

In [23]:
# create, update, set
collection.document('goodnight').set({'test9': 900})

update_time {
  seconds: 1763791225
  nanos: 67703000
}

In [31]:
us = [u.to_dict() for u in collection.stream()]

In [34]:
us

[{'follow': '2025/11/22-14:44:46',
  'language': 'zh-TW',
  'status_message': None,
  'display_name': 'Enos',
  'picture_url': 'https://sprofile.line-scdn.net/0hRChPEC5sDWxFKSb35qhzEzV5DgZmWFR-bxxGCScvVg9xGEs5O0ZHWiQgUlgtGk07Pk4WCiN8Bl1JOnoKW3_xWEIZU1t_GUo8YUlLiw',
  'weather_count': 3,
  'unfollow': '2025/11/22-14:44:34',
  'user_id': 'U091e03d020e37cce02ba9fd2f03377ec'},
 {'follow': '2025/11/22-15:11:38',
  'language': 'zh-TW',
  'status_message': None,
  'display_name': 'Leo Huang',
  'user_id': 'U1342a54b5692145c89b24338c1708db7',
  'picture_url': 'https://sprofile.line-scdn.net/0hW0vATS7eCB9bOBoGiUV2YCtoC3V4SVENcg4TKjw7BS9iDhpPJFtEfD06UigzC0lIIFhGcGw-UnxXK395RW70K1wIVihhCE9Pf1hO-A'},
 {'follow': '2025/11/22-15:10:15',
  'language': 'zh-TW',
  'status_message': None,
  'display_name': '陳品蓁',
  'user_id': 'U601868d1f9414f4884e74584dcec6b3c',
  'picture_url': 'https://sprofile.line-scdn.net/0hClB1RzLNHENvSAM6ANRifR8YHylMOUVRSipRcF1BQidSf1JCEShWIQ9KRXIFLVsdRXsDLVsdRnJNfjpHQVkFXSQ1GCsu

In [2]:
import time

t = 1762579212668
time.localtime(t/1000)

time.struct_time(tm_year=2025, tm_mon=11, tm_mday=8, tm_hour=13, tm_min=20, tm_sec=12, tm_wday=5, tm_yday=312, tm_isdst=0)

In [49]:
a = {'a': 100, 'b': {'x': [200, 100]}}
c = a.copy()

In [ ]:
impor

In [50]:
c['b'] = {'x': [300]}

In [51]:
c

{'a': 100, 'b': {'x': [300]}}

In [52]:
a

{'a': 100, 'b': {'x': [200, 100]}}

In [ ]:
!pip install pillow

In [22]:
from PIL import Image

image = Image.open('test.jpg')
image.size

(960, 1706)

In [23]:
image.height

1706

In [26]:
image.width

960

In [ ]:
960 x 1706

In [11]:
import time

int(time.time())+60

1763185395

In [ ]:
!pip install google-cloud-storage

In [5]:
from google.cloud import storage

storage_client = storage.Client.from_service_account_json('service.json')

In [6]:
bucket = storage_client.bucket('myfirstprojectnumberonebucket')

In [12]:
blob = bucket.blob('bbb.jpg')
blob.upload_from_filename('donav.jpg')

In [13]:
blob.generate_signed_url(int(time.time())+60)

'https://storage.googleapis.com/myfirstprojectnumberonebucket/bbb.jpg?Expires=1763185567&GoogleAccessId=myfirstserviceaccount%40firm-braid-476903-s2.iam.gserviceaccount.com&Signature=oVP7UhKydbTG8PktG%2FXmH1FOsd8%2F2g50PMaiCRTqP8kxlCwKvg2maYRPZvuE4%2BE7xi5A0kr3pBvQd8zMlQINVymLmubAwq9MG1AvOa2E8YkR3j8C%2F0TrI%2FlbSDUJAXpaS97kP7FQSqNTQvlqVJGjZfBI%2B78xEZea7YJ%2Fx5VxsSyx0Ehum2LW%2B15MwUkFFT4%2Bc%2BRNtwYcquv%2BtQ%2Fjv3VX56nCwQCIeovhTeoVO5cEreMin3HJ5alVgOfDnohZ3tGWwUCGLFpFPmM23L7jKpMvGT1TawVcxM2fNUJqUURXMU0mTV%2FsVLJ8Dp0FDrYPBrt2nkrx2Rz9YcDWPjlAUdQlAQ%3D%3D'

In [35]:
bucket.blob('abc.jpg').upload_from_filename('donav.jpg')

In [ ]:
help(blob.upload_from_string)

In [20]:
with open('me.json', encoding='utf-8') as f:
    me = json.load(f)

In [27]:
me

{'type': 'bubble',
 'header': {'type': 'box',
  'layout': 'baseline',
  'contents': [{'type': 'text',
    'text': '粉絲一覽',
    'size': '4xl',
    'align': 'center',
    'color': '#FF0000'}]},
 'hero': {'type': 'image',
  'url': 'https://cdn.britannica.com/94/256194-050-DD861124/Shohei-Ohtani-Los-Angeles-Dodgers.jpg',
  'size': 'full',
  'aspectRatio': '4:3',
  'action': {'type': 'uri', 'uri': 'https://line.me/'}},
 'body': {'type': 'box',
  'layout': 'vertical',
  'contents': [{'type': 'box',
    'layout': 'vertical',
    'margin': 'lg',
    'spacing': 'sm',
    'contents': [{'type': 'box',
      'layout': 'baseline',
      'spacing': 'sm',
      'contents': [{'type': 'text',
        'text': 'User ID',
        'color': '#0000FF',
        'size': 'sm',
        'flex': 0},
       {'type': 'text',
        'text': 'U12345678901234567890123456789012',
        'wrap': False,
        'color': '#666666',
        'size': 'sm',
        'flex': 5}]},
     {'type': 'box',
      'layout': 'baseline'

In [25]:
car = {"type": "carousel", "contents": [me, me]}

In [26]:
car

{'type': 'carousel',
 'contents': [{'type': 'bubble',
   'header': {'type': 'box',
    'layout': 'baseline',
    'contents': [{'type': 'text',
      'text': '粉絲一覽',
      'size': '4xl',
      'align': 'center',
      'color': '#FF0000'}]},
   'hero': {'type': 'image',
    'url': 'https://cdn.britannica.com/94/256194-050-DD861124/Shohei-Ohtani-Los-Angeles-Dodgers.jpg',
    'size': 'full',
    'aspectRatio': '4:3',
    'action': {'type': 'uri', 'uri': 'https://line.me/'}},
   'body': {'type': 'box',
    'layout': 'vertical',
    'contents': [{'type': 'box',
      'layout': 'vertical',
      'margin': 'lg',
      'spacing': 'sm',
      'contents': [{'type': 'box',
        'layout': 'baseline',
        'spacing': 'sm',
        'contents': [{'type': 'text',
          'text': 'User ID',
          'color': '#0000FF',
          'size': 'sm',
          'flex': 0},
         {'type': 'text',
          'text': 'U12345678901234567890123456789012',
          'wrap': False,
          'color': '#66666

In [21]:
me

{'type': 'bubble',
 'header': {'type': 'box',
  'layout': 'baseline',
  'contents': [{'type': 'text',
    'text': '粉絲一覽',
    'size': '4xl',
    'align': 'center',
    'color': '#FF0000'}]},
 'hero': {'type': 'image',
  'url': 'https://cdn.britannica.com/94/256194-050-DD861124/Shohei-Ohtani-Los-Angeles-Dodgers.jpg',
  'size': 'full',
  'aspectRatio': '4:3',
  'action': {'type': 'uri', 'uri': 'https://line.me/'}},
 'body': {'type': 'box',
  'layout': 'vertical',
  'contents': [{'type': 'box',
    'layout': 'vertical',
    'margin': 'lg',
    'spacing': 'sm',
    'contents': [{'type': 'box',
      'layout': 'baseline',
      'spacing': 'sm',
      'contents': [{'type': 'text',
        'text': 'User ID',
        'color': '#0000FF',
        'size': 'sm',
        'flex': 0},
       {'type': 'text',
        'text': 'U12345678901234567890123456789012',
        'wrap': False,
        'color': '#666666',
        'size': 'sm',
        'flex': 5}]},
     {'type': 'box',
      'layout': 'baseline'